In [45]:
import pandas as pd

rcp = pd.read_csv("C:/project/company/yk_cmp_rpt.csv")

In [140]:
rcp = ['20120419000097']

In [46]:
fin = pd.concat(concat_list, sort=False , axis =0)
fin.to_excel("C:/project/vv/a652.xlsx")

27674

In [ ]:
from html_table_parser import parser_functions as parser
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import re

# 재무제표 정규표현식 #
fin_stat_code1 = re.compile("현[ \s]*금[ \s]*및[ \s]*((예[ \s]*치[ \s]*금)|(현[ \s]*금[ \s]*((성[ \s]*자[ \s]*산)|(등[ \s]*가[ \s]*물))))")
fin_stat_code2 = re.compile("자[ \s]*본[ \s]*총[ \s]*계")
fin_stat_code3 = re.compile("자[ \s]*산[ \s]*총[ \s]*계")
 
# 재무제표 금액단위 #
re_unit1 = re.compile('단[\s]*위[ \s]*:[ \s]*원') # 원
re_unit2 = re.compile('단[\s]*위[ \s]*:[ \s]*천[\s]*') # 천원
re_unit3 = re.compile('단[\s]*위[ \s]*:[ \s]*백[\s]*만[\s]*') # 백만원
re_unit4 = re.compile('단[\s]*위[ \s]*:[ \s]*C[ \s]*N[ \s]*Y[ \s]*') # 위안
re_unit5 = re.compile('단[\s]*위[ \s]*:[ \s]*R[ \s]*M[ \s]*B[ \s]*') # 위안
re_unit6 = re.compile('단[\s]*위[ \s]*:[ \s]*C[ \s]*N[ \s]*H[ \s]*') # 위안
re_unit7 = re.compile('단[\s]*위[ \s]*:[ \s]*J[ \s]*P[ \s]*Y[ \s]*') # 엔
re_unit8 = re.compile('단[\s]*위[ \s]*:[ \s]*U[ \s]*S[ \s]*D[ \s]*') # 달러
re_unit9 = re.compile('단[\s]*위[ \s]*:[ \s]*억[ \s]*원[\s]*')


concat_list = []
#보고서 번호 입력 #
for idx, rep_num in enumerate(rcp['rcp_no'][114867:]):
#for idx, rep_num in enumerate(rcp):
    #if idx % 70 ==0:
    print(idx, '번째 보고서 번호는 :', rep_num , '입니다 .')
              
    # URL, CONTENT #
    rep_url      = """http://dart.fss.or.kr/dsaf001/main.do?rcpNo={}""".format(rep_num)
    rep_page     = requests.get(rep_url)
    rep_menu     = BeautifulSoup(rep_page.content,'html.parser')
    rep_menu_txt = rep_menu.text

    
    # 재무제표 위치값 찾기 #
    key1 = rep_menu_txt.find("2. 연결재무제표")    ; key2 = rep_menu_txt.find("4. 연결재무제표")
    key3 = rep_menu_txt.find("4. 재무제표")        ; key4 = rep_menu_txt.find("재무제표 등")
    key5 = rep_menu_txt.find("재무에 관한")        ; key6 = rep_menu_txt.find('3. 재무제표')
    key7 = rep_menu_txt.find('연 결 재 무 제 표')  ; key8 = rep_menu_txt.find('반 기 재 무 제 표')
    
    key_basket = [ key1, key2, key3, key4, key5, key6, key7, key8 ]
    key_basket = list(filter(lambda x: x != -1 , key_basket))
    
    # 키워드가 없어서 다음 보고서로 넘어감 #
    if len(key_basket) == 0 :
        continue
        
    for key in key_basket:
        x = rep_menu_txt[key:key+170] ; x = x[:x.find('d\'')+2]
        
        # URL 데이터 추출
        raw_dcm = x.split(',')[4] ; raw_ele = x.split(',')[5] ; raw_off = x.split(',')[6]
        raw_leg = x.split(',')[7] ; raw_dtd = x.split(',')[8].replace("\'","")
        
        dcm = raw_dcm.replace("\'","").strip()
        ele = raw_ele.replace("\'","").strip()
        off = raw_off.replace("\'","").strip()
        leg = raw_leg.replace("\'","").strip()
        dtd = raw_dtd.replace("\'","").strip()

        fin_stat_url = """http://dart.fss.or.kr/report/viewer.do"""
        fin_stat_par = {'rcpNo':rep_num ,'dcmNo': dcm,'eleId': ele,'offset': off,'length': leg,'dtd': dtd}

        fin_stat_page = requests.get(fin_stat_url, fin_stat_par)
        fin_stat_rslt = BeautifulSoup(fin_stat_page.content,'html.parser')

        check_table = fin_stat_rslt.find_all('table')
        check_unit =  fin_stat_rslt.find_all('body')[0].text[:700]

#----------------------------------------------------------------------------------------------------------------------------- 
        
        if len(check_table) < 2:
            continue
        
        table_loc = 0
        cnt       = 0
        for table in check_table:
            if fin_stat_code1.search(table.text):
                table_loc = cnt
                break
            elif fin_stat_code2.search(table.text):
                table_loc = cnt
                break
            elif fin_stat_code3.search(table.text):
                table_loc = cnt
                break
            cnt = cnt + 1
        
        balance_table = check_table[table_loc]
        unit_table = check_table[table_loc-1]
        
#-------단위금액--------------------------------------------------------------------------------------------------------------     
        
        unit = 0
        if len(unit_table(string=re_unit1)) != 0:
            unit = 1
        elif len(unit_table(string=re_unit2)) != 0:
            unit = 2
        elif len(unit_table(string=re_unit3)) != 0:
            unit = 3
        elif len(unit_table(string=re_unit4)) != 0:
            unit = 4
        elif len(unit_table(string=re_unit5)) != 0:
            unit = 5
        elif len(unit_table(string=re_unit6)) != 0:
            unit = 6
        elif len(unit_table(string=re_unit7)) != 0:
            unit = 7
        elif len(unit_table(string=re_unit8)) != 0:
            unit = 8
        elif len(unit_table(string=re_unit9)) != 0:
            unit = 9
            
        if unit ==0:
            if re_unit1.search(check_unit):
                unit = 1
            elif re_unit2.search(check_unit):
                unit = 2
            elif re_unit3.search(check_unit):
                unit = 3
            elif re_unit4.search(check_unit):
                unit = 4
            elif re_unit5.search(check_unit):
                unit = 5
            elif re_unit6.search(check_unit):
                unit = 6
            elif re_unit7.search(check_unit):
                unit = 7
            elif re_unit8.search(check_unit):
                unit = 8
            elif re_unit9.search(check_unit):
                unit = 9

#-----------------------------------------------------------------------------------------------------------------------------
        table_2d    = parser.make2d(balance_table)
        fin_stat_df = pd.DataFrame(table_2d)
        fin_stat_df = fin_stat_df.fillna('')
        
        if len(fin_stat_df.columns) < 2 or len(fin_stat_df[0]) < 14:
            continue
            
        fin_stat_df.replace(regex=['[\s]+','[.]+\.'], value='', inplace=True)
        
        if len(fin_stat_df.isin(['주석'])[1].value_counts()) == 2:
            fin_stat_df.drop(1, axis=1, inplace=True)
            
        
        fin_stat_df.drop_duplicates(0,inplace=True, keep='first')
        fin_stat_df.reset_index(drop=True,inplace=True)
        
        fin_stat_df[0].replace(regex=['\(.+\)'], value='', inplace=True)
        fin_stat_df.iloc[:2,:] = fin_stat_df.iloc[:2,:].replace(regex=['\(.+\)'], value='')
        fin_stat_df.iloc[2:,1:] = fin_stat_df.iloc[2:,1:].replace(regex=['[()]'], value='')
        
        fin_stat_df.iloc[0,0] = '과목'
        fin_stat_df[0].replace(regex=['[^가-힣]'],value='',inplace=True)
        
        dump_name = []
        for idx, col in enumerate(fin_stat_df[0]):
            if len(col) >= 9 or len(col)==0:
                dump_name.append(idx)
                
        fin_stat_df.drop(dump_name, 0, inplace=True)
        fin_stat_df.reset_index(drop=True,inplace=True)
        
        #fin_stat_df[0].replace(regex=['[^가-힣]'],value='',inplace=True)

        if len(fin_stat_df.columns) == 2:
            
            dump_col = []
            for idx, col in enumerate(fin_stat_df.iloc[:,1]):
                if len(col) >= 19 :
                    dump_col.append(idx)

            fin_stat_df.drop(dump_col, 0, inplace=True)
            fin_stat_df.reset_index(drop=True,inplace=True)
            
            fin_stat_df.iloc[1:,1:].replace(regex=['△', 'Δ'], value = '-', inplace=True)
            #fin_stat_df.iloc[1:,1:].replace(regex=['\(.+\)'], value = '', inplace =True)
            fin_stat_df.iloc[1:,1:].replace(regex=['[^-0-9]'], value = '', inplace =True)
            #fin_stat_df.iloc[1:,1:].replace(regex=['-$'], value = '', inplace =True)
            #fin_stat_df.iloc[1:,1:].replace(regex=['--+'], value = '-', inplace =True)
            fin_stat_df.iloc[1:,1:].replace(to_replace=['','-','0'], value = np.nan,inplace=True)

            result_df = fin_stat_df
            result_df.dropna(inplace=True)
            
        
        elif fin_stat_df.iloc[0,1] == fin_stat_df.iloc[0,2]:
            try:
                dump_col1 = []
                for idx, col in enumerate(fin_stat_df.iloc[:,1]):
                    if len(col) >= 19 :
                        dump_col.append(idx)

                fin_stat_df.drop(dump_col1, 0, inplace=True)
                fin_stat_df.reset_index(drop=True,inplace=True)


                dump_col2 = []
                for idx, col in enumerate(fin_stat_df.iloc[:,2]):
                    if len(col) >= 19 :
                        dump_col.append(idx)

                fin_stat_df.drop(dump_col2, 0, inplace=True)
                fin_stat_df.reset_index(drop=True,inplace=True)
                #try:

                fin_stat_df = fin_stat_df.iloc[:,:3]

                fin_stat_df.iloc[1:,1:].replace(regex=['△', 'Δ'], value = '-', inplace =True)
                #fin_stat_df.iloc[1:,1:].replace(regex=['\(.+\)'], value = '', inplace =True)
                fin_stat_df.iloc[1:,1:].replace(regex=['[^-0-9]'], value = '', inplace =True)
                fin_stat_df.iloc[1:,1:].replace(regex=['--+'], value = '-', inplace =True)
                fin_stat_df.iloc[1:,1:].replace(regex=['-$'], value = '', inplace =True)
                fin_stat_df.iloc[1:,1:].replace(to_replace=['','-','0'], value = 0,inplace=True)

                fin_stat_df.iloc[1:,1:] = fin_stat_df.iloc[1:,1:].astype('float64')


                left = fin_stat_df[0] ; fin_stat_df.iloc[0,2] = ''
                right = fin_stat_df.iloc[:,1] + fin_stat_df.iloc[:,2]

                result_df = pd.DataFrame({0:left, 1:right})
                result_df.replace(to_replace=[0],value=np.nan, inplace=True)
                result_df.dropna(inplace=True)
            except:
                continue
        
        elif fin_stat_df.iloc[0,2] == '':
            try:
                dump_col1 = []
                for idx, col in enumerate(fin_stat_df.iloc[:,1]):
                    if len(col) >= 19 :
                        dump_col.append(idx)

                fin_stat_df.drop(dump_col1, 0, inplace=True)
                fin_stat_df.reset_index(drop=True,inplace=True)


                dump_col2 = []
                for idx, col in enumerate(fin_stat_df.iloc[:,2]):
                    if len(col) >= 19 :
                        dump_col.append(idx)

                fin_stat_df.drop(dump_col2, 0, inplace=True)
                fin_stat_df.reset_index(drop=True,inplace=True)


                fin_stat_df = fin_stat_df.iloc[:,:3]

                fin_stat_df.iloc[1:,1:].replace(regex=['△', 'Δ'], value = '-', inplace =True)
                #fin_stat_df.iloc[1:,1:].replace(regex=['\(.+\)'], value = '', inplace =True)
                fin_stat_df.iloc[1:,1:].replace(regex=['[^-0-9]'], value = '', inplace =True)
                #fin_stat_df.iloc[1:,1:].replace(regex=['--+'], value = '-', inplace =True)
                #fin_stat_df.iloc[1:,1:].replace(regex=['-$'], value = '', inplace =True)
                fin_stat_df.iloc[1:,1:].replace(to_replace=['','-','0'], value = 0,inplace=True)

                fin_stat_df.iloc[1:,1:] = fin_stat_df.iloc[1:,1:].astype('int64')


                left = fin_stat_df[0] ; fin_stat_df.iloc[0,2] = ''
                right = fin_stat_df.iloc[:,1] + fin_stat_df.iloc[:,2]

                result_df = pd.DataFrame({0:left, 1:right})
                result_df.replace(to_replace=[0],value=np.nan, inplace=True)
                result_df.dropna(inplace=True)
            except:
                continue
            
        else:
            fin_stat_df = fin_stat_df.iloc[:,:2]

            fin_stat_df.iloc[1:,1:].replace(regex=['△', 'Δ'], value = '-', inplace=True)
            #fin_stat_df.iloc[1:,1:].replace(regex=['\(.+\)'], value = '', inplace =True)
            fin_stat_df.iloc[1:,1:].replace(regex=['[^-0-9]'], value = '', inplace =True)
            #fin_stat_df.iloc[1:,1:].replace(regex=['--+'], value = '-', inplace =True)
            #fin_stat_df.iloc[1:,1:].replace(regex=['-$'], value = '', inplace =True)
            fin_stat_df.iloc[1:,1:].replace(to_replace=['','-','0'], value = np.nan,inplace=True)
            result_df = fin_stat_df
            result_df.dropna(inplace=True)
            
        
        
        result_df.drop_duplicates(0, inplace=True, keep='first')
        result_df.reset_index(drop=True, inplace=True)
        result_df = result_df.set_index(0).T
        
        # 데이터 추가 #
      
        if len(result_df.columns) > 11:
            result_df['rcp_no'] = rep_num
            result_df['unit'] = unit
            concat_list.append(result_df)
            break
        
        else:
            continue
            

            
fin = pd.concat(concat_list, sort=False , axis =0)
fin.to_excel("C:/project/vv/ae.xlsx")